In [3]:
pip install open3d

  Using cached open3d-0.19.0-cp312-cp312-win_amd64.whl.metadata (4.2 kB)
  Using cached dash-3.2.0-py3-none-any.whl.metadata (10 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached flask-3.1.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached configargparse-1.7.1-py3-none-any.whl.metadata (24 kB)
  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached plotly-6.3.1-py3-none-any.whl.metadata (8.5 kB)
  Using cached importlib_metadata-8.7.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached retrying-1.4.2-py3-none-any.whl.metadata (5.5 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
  Using cached fastjsonschema-2.21.2-py3-none-any.whl.metadata (2.3 kB)
  Using cached zipp-3.23.0-py3-none-any.wh


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import open3d as o3d
import cv2
import numpy as np

# --- 1. ĐỊNH NGHĨA CÁC THAM SỐ CAMERA (Lấy từ đề bài) ---
# Tham số nội tại camera màu
color_intrinsics_params = {
    "width": 1280, "height": 720,
    "fx": 643.90087890625, "fy": 643.1365356445312,
    "cx": 650.2113037109375, "cy": 355.79559326171875
}

# Tham số nội tại camera độ sâu
depth_intrinsics_params = {
    "width": 1280, "height": 720,
    "fx": 650.0616455078125, "fy": 650.0616455078125,
    "cx": 649.5928955078125, "cy": 360.9415588378906
}

# Tham số ngoại tại: Chuyển từ Depth sang Color
# Đề bài không cung cấp R và t, đây là một điểm thiếu sót.
# Giả sử R là ma trận đơn vị và t là vector 0 (trường hợp lý tưởng khi 2 camera trùng nhau)
# TRONG THỰC TẾ, BẠN CẦN CÓ MA TRẬN NÀY.
# extrinsic_depth_to_color = np.identity(4) # Ma trận biến đổi 4x4

# --- 2. ĐỌC DỮ LIỆU ẢNH ---
# Thay thế bằng đường dẫn thực tế của bạn
color_raw = cv2.imread("./train/rgb/0001.png")
depth_raw = cv2.imread("./train/depth/0001.png", cv2.IMREAD_UNCHANGED)

# Chuyển ảnh sang định dạng của Open3D
# OpenCV đọc ảnh BGR, cần chuyển sang RGB
color_o3d = o3d.geometry.Image(cv2.cvtColor(color_raw, cv2.COLOR_BGR2RGB))
depth_o3d = o3d.geometry.Image(depth_raw)

# --- 3. CĂN CHỈNH DỮ LIỆU ---
# Tạo đối tượng RGBDImage. Open3D dùng điều này để kết hợp màu và độ sâu.
# depth_scale=1000.0 giả định rằng giá trị pixel trong ảnh depth là milimet. 
# 1500 / 1000.0 = 1.5 mét.
# depth_trunc=3.0 nghĩa là các vật ở xa hơn 3 mét sẽ bị bỏ qua.
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_o3d,
    depth_o3d,
    depth_scale=1000.0, # QUAN TRỌNG: Kiểm tra lại đơn vị của ảnh depth
    depth_trunc=3.0,   # Cắt bỏ các điểm ở quá xa
    convert_rgb_to_intensity=False
)

# --- 4. TẠO POINT CLOUD ĐÃ ĐƯỢC CĂN CHỈNH ---
# Sử dụng tham số NỘI TẠI CỦA CAMERA MÀU để tạo point cloud.
# Điều này đảm bảo point cloud được tạo ra trong hệ tọa độ của camera màu.
camera_intrinsics = o3d.camera.PinholeCameraIntrinsic(
    width=color_intrinsics_params["width"],
    height=color_intrinsics_params["height"],
    fx=color_intrinsics_params["fx"],
    fy=color_intrinsics_params["fy"],
    cx=color_intrinsics_params["cx"],
    cy=color_intrinsics_params["cy"]
)

# Tạo point cloud từ RGBDImage
# Ma trận ngoại tại ở đây là ma trận camera-to-world, nếu không có thì mặc định là identity.
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image,
    camera_intrinsics
)

# (Tùy chọn) Lật point cloud nếu nó bị ngược
# pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

# Hiển thị để kiểm tra
o3d.visualization.draw_geometries([pcd])



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import numpy as np
import matplotlib.pyplot as plt

# --- 1. TÌM MẶT PHẲNG BẰNG RANSAC ---
plane_model, inlier_indices = pcd.segment_plane(
    distance_threshold=0.01,  # Ngưỡng khoảng cách 1cm
    ransac_n=3,
    num_iterations=1000
)

# In ra phương trình mặt phẳng Ax + By + Cz + D = 0
# [a, b, c] là vector pháp tuyến của mặt phẳng
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")


# --- 2. TÁCH BIỆT BĂNG TẢI VÀ BƯU KIỆN ---

# Lấy các điểm thuộc mặt phẳng (băng tải)
inlier_cloud = pcd.select_by_index(inlier_indices)
inlier_cloud.paint_uniform_color([1.0, 0, 0]) # Tô màu đỏ cho băng tải

# Lấy các điểm không thuộc mặt phẳng (bưu kiện)
# Tham số invert=True để lấy những chỉ số không có trong danh sách inlier_indices
outlier_cloud = pcd.select_by_index(inlier_indices, invert=True)
outlier_cloud.paint_uniform_color([0, 1.0, 0]) # Tô màu xanh cho bưu kiện

# Hiển thị để kiểm tra trực quan
print("Hiển thị băng tải (màu đỏ) và các vật thể (màu xanh)")
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

# Giờ 'outlier_cloud' chính là point cloud chỉ chứa các bưu kiện
parcels_pcd = outlier_cloud

Plane equation: 0.01x + -0.00y + 1.00z + -1.19 = 0
Hiển thị băng tải (màu đỏ) và các vật thể (màu xanh)


In [3]:
# --- 3. PHÂN CỤM CÁC BƯU KIỆN BẰNG DBSCAN ---

# Chạy thuật toán DBSCAN
labels = np.array(parcels_pcd.cluster_dbscan(
    eps=0.02,           # Khoảng cách 2cm
    min_points=10,
    print_progress=True
))

# Lấy số lượng cụm đã tìm thấy (nhãn -1 là nhiễu)
max_label = labels.max()
print(f"Tìm thấy {max_label + 1} cụm bưu kiện.")

# Tô màu cho từng cụm để dễ nhận biết
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0  # Điểm nhiễu (label = -1) tô màu đen
parcels_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

print("Hiển thị các bưu kiện được phân cụm theo màu")
o3d.visualization.draw_geometries([parcels_pcd])

# 'labels' là một mảng, mỗi phần tử tương ứng với một điểm trong parcels_pcd
# và cho biết điểm đó thuộc về cụm nào (0, 1, 2, ...).
# Bây giờ bạn có thể lặp qua từng label để xử lý riêng từng bưu kiện.

Tìm thấy 195 cụm bưu kiện.
Hiển thị các bưu kiện được phân cụm theo màu


In [4]:
# 1. Tạo một dictionary để lưu trữ point cloud của từng bưu kiện riêng lẻ
# Key sẽ là nhãn (0, 1, 2,...), value là đối tượng PointCloud tương ứng
individual_parcels = {}

# 2. Lấy tất cả các nhãn cụm duy nhất có trong mảng labels
# Thuật toán DBSCAN gán nhãn -1 cho các điểm nhiễu (noise), nên chúng ta cần loại bỏ nó.
unique_labels = set(labels)
if -1 in unique_labels:
    unique_labels.remove(-1)

# 3. Lặp qua từng nhãn duy nhất để trích xuất point cloud tương ứng
print(f"🔎 Bắt đầu tách {len(unique_labels)} bưu kiện đã được phân cụm...")

for label_id in unique_labels:
    # Lấy ra tất cả các chỉ số (indices) của những điểm thuộc về label hiện tại
    indices = np.where(labels == label_id)[0]

    # Dùng các chỉ số này để chọn ra các điểm từ point cloud gốc
    parcel_pcd = parcels_pcd.select_by_index(indices)

    # Lưu point cloud của bưu kiện vừa tách vào dictionary
    individual_parcels[label_id] = parcel_pcd
    
    print(f"  ✅ Đã tách bưu kiện có nhãn {label_id}, bao gồm {len(parcel_pcd.points)} điểm.")

# --- 4. Sử dụng kết quả ---
# Giờ đây, có thể dễ dàng truy cập và xử lý từng bưu kiện một.
# Ví dụ: In ra tâm của tất cả các bưu kiện đã tìm thấy.
print("\n--- Tọa độ tâm của từng bưu kiện ---")
for label_id, pcd in individual_parcels.items():
    center_point = pcd.get_center()
    print(f"Bưu kiện {label_id}: Tọa độ tâm = {center_point}")

# (Tùy chọn) Hiển thị một bưu kiện cụ thể, ví dụ bưu kiện có nhãn 0
if 0 in individual_parcels:
    print("\nHiển thị bưu kiện có nhãn số 0...")
    target_parcel = individual_parcels[0]
    target_parcel.paint_uniform_color([0.8, 0.2, 0.2]) # Tô màu đỏ cho dễ thấy
    o3d.visualization.draw_geometries([target_parcel])


🔎 Bắt đầu tách 195 bưu kiện đã được phân cụm...
  ✅ Đã tách bưu kiện có nhãn 0, bao gồm 10889 điểm.
  ✅ Đã tách bưu kiện có nhãn 1, bao gồm 6372 điểm.
  ✅ Đã tách bưu kiện có nhãn 2, bao gồm 29103 điểm.
  ✅ Đã tách bưu kiện có nhãn 3, bao gồm 23119 điểm.
  ✅ Đã tách bưu kiện có nhãn 4, bao gồm 15890 điểm.
  ✅ Đã tách bưu kiện có nhãn 5, bao gồm 199876 điểm.
  ✅ Đã tách bưu kiện có nhãn 6, bao gồm 148 điểm.
  ✅ Đã tách bưu kiện có nhãn 7, bao gồm 274 điểm.
  ✅ Đã tách bưu kiện có nhãn 8, bao gồm 44 điểm.
  ✅ Đã tách bưu kiện có nhãn 9, bao gồm 217 điểm.
  ✅ Đã tách bưu kiện có nhãn 10, bao gồm 82 điểm.
  ✅ Đã tách bưu kiện có nhãn 11, bao gồm 986 điểm.
  ✅ Đã tách bưu kiện có nhãn 12, bao gồm 12 điểm.
  ✅ Đã tách bưu kiện có nhãn 13, bao gồm 23 điểm.
  ✅ Đã tách bưu kiện có nhãn 14, bao gồm 62 điểm.
  ✅ Đã tách bưu kiện có nhãn 15, bao gồm 604 điểm.
  ✅ Đã tách bưu kiện có nhãn 16, bao gồm 186 điểm.
  ✅ Đã tách bưu kiện có nhãn 17, bao gồm 56 điểm.
  ✅ Đã tách bưu kiện có nhãn 18, bao g

In [5]:
# --- 1. CÁC THAM SỐ CẦN THIẾT ---
# Tham số nội tại camera màu (quan trọng cho việc chiếu)
color_intrinsics_params = {
    "fx": 643.90087890625, "fy": 643.1365356445312,
    "cx": 650.2113037109375, "cy": 355.79559326171875
}
fx = color_intrinsics_params["fx"]
fy = color_intrinsics_params["fy"]
cx = color_intrinsics_params["cx"]
cy = color_intrinsics_params["cy"]

# ROI băng tải 2 theo đề bài (X, Y, W, H)
ROI = (560, 150, 300, 330)
roi_x, roi_y, roi_w, roi_h = ROI

In [6]:
# --- 2. LỌC CÁC BƯU KIỆN ỨNG VIÊN ---

# Tạo một danh sách để lưu các bưu kiện hợp lệ (nằm trong ROI)
# Mỗi phần tử sẽ là một dictionary chứa thông tin cần thiết cho bước sau
candidate_parcels = []

print("🔎 Bắt đầu lọc bưu kiện theo ROI...")
for label_id, pcd in individual_parcels.items():
    # 1. Tính tâm 3D của bưu kiện
    center_3d = pcd.get_center()
    X, Y, Z = center_3d
    
    # 2. Chiếu tâm 3D về tọa độ pixel 2D
    # Z không thể bằng 0
    if Z == 0:
        continue
        
    u = int((X * fx / Z) + cx)
    v = int((Y * fy / Z) + cy)
    
    # 3. Kiểm tra xem tọa độ pixel (u, v) có nằm trong ROI không
    is_in_roi = (roi_x <= u < roi_x + roi_w) and \
                (roi_y <= v < roi_y + roi_h)
    
    print(f"Bưu kiện {label_id}: Tâm 3D {np.round(center_3d, 2)} -> Chiếu về 2D ({u}, {v})", end="")
    
    # 4. Nếu nằm trong ROI, lưu nó vào danh sách ứng viên
    if is_in_roi:
        print(" -> ✅ TRONG ROI")
        candidate_parcels.append({
            "label": label_id,
            "pcd": pcd,
            "center_3d": center_3d
        })
    else:
        print(" -> ❌ NGOÀI ROI")

# In ra kết quả
print(f"\n🎉 Đã tìm thấy {len(candidate_parcels)} bưu kiện ứng viên nằm trong ROI.")

🔎 Bắt đầu lọc bưu kiện theo ROI...
Bưu kiện 0: Tâm 3D [-0.6  -0.88  1.81] -> Chiếu về 2D (436, 43) -> ❌ NGOÀI ROI
Bưu kiện 1: Tâm 3D [-0.43 -1.07  2.4 ] -> Chiếu về 2D (534, 70) -> ❌ NGOÀI ROI
Bưu kiện 2: Tâm 3D [ 0.21 -1.11  2.32] -> Chiếu về 2D (708, 47) -> ❌ NGOÀI ROI
Bưu kiện 3: Tâm 3D [ 0.49 -0.45  1.34] -> Chiếu về 2D (885, 138) -> ❌ NGOÀI ROI
Bưu kiện 4: Tâm 3D [ 0.84 -0.55  1.15] -> Chiếu về 2D (1122, 46) -> ❌ NGOÀI ROI
Bưu kiện 5: Tâm 3D [-0.33 -0.03  0.89] -> Chiếu về 2D (411, 332) -> ❌ NGOÀI ROI
Bưu kiện 6: Tâm 3D [-0.43 -0.29  0.54] -> Chiếu về 2D (131, 7) -> ❌ NGOÀI ROI
Bưu kiện 7: Tâm 3D [-0.97 -0.59  1.12] -> Chiếu về 2D (92, 15) -> ❌ NGOÀI ROI
Bưu kiện 8: Tâm 3D [ 0.98 -0.63  1.19] -> Chiếu về 2D (1179, 12) -> ❌ NGOÀI ROI
Bưu kiện 9: Tâm 3D [-2.72 -1.51  2.89] -> Chiếu về 2D (43, 19) -> ❌ NGOÀI ROI
Bưu kiện 10: Tâm 3D [-0.49 -0.76  1.46] -> Chiếu về 2D (435, 20) -> ❌ NGOÀI ROI
Bưu kiện 11: Tâm 3D [-0.43 -0.24  0.52] -> Chiếu về 2D (119, 64) -> ❌ NGOÀI ROI
Bưu kiện 12: T

In [7]:
# --- 1. KIỂM TRA ĐIỀU KIỆN ĐẦU VÀO ---
if not candidate_parcels:
    print("❌ Không có bưu kiện ứng viên nào. Dừng xử lý.")
    # Thoát hoặc xử lý trường hợp không có bưu kiện
    target_parcel = None
else:
    # --- 2. SẮP XẾP CÁC BƯU KIỆN THEO CHIỀU CAO ---
    # Sắp xếp danh sách dựa trên tọa độ Z của 'center_3d'.
    # `lambda item: item['center_3d'][2]` ra lệnh cho hàm sort dùng giá trị Z làm khóa so sánh.
    # Z càng nhỏ -> càng cao -> xếp trước.
    candidate_parcels.sort(key=lambda item: item['center_3d'][2])

    print("📦 Danh sách bưu kiện ứng viên sau khi sắp xếp theo độ cao (Z tăng dần):")
    for p in candidate_parcels:
        print(f"  - Bưu kiện {p['label']}: Z = {p['center_3d'][2]:.4f}")

    # --- 3. ÁP DỤNG LUẬT CHỌN MỤC TIÊU ---
    # Bưu kiện cao nhất chắc chắn là phần tử đầu tiên trong danh sách đã sắp xếp
    highest_parcel = candidate_parcels[0]
    
    # Tạo một nhóm chứa các bưu kiện có độ cao tương đương (chênh lệch < 5mm)
    top_group = [highest_parcel]
    highest_z = highest_parcel['center_3d'][2]

    # Duyệt qua các bưu kiện còn lại
    for i in range(1, len(candidate_parcels)):
        current_parcel = candidate_parcels[i]
        current_z = current_parcel['center_3d'][2]
        
        # Kiểm tra chênh lệch độ cao (tính bằng mét)
        if current_z - highest_z < 0.005: # 5mm
            top_group.append(current_parcel)
        else:
            # Vì danh sách đã được sắp xếp, nếu chênh lệch đã > 5mm thì không cần xét tiếp
            break

    # --- 4. XỬ LÝ TIE-BREAKING NẾU CẦN ---
    if len(top_group) == 1:
        # Nếu chỉ có 1 bưu kiện trong nhóm cao nhất -> chọn luôn
        target_parcel = top_group[0]
        print(f"\n✨ Chỉ có một bưu kiện cao nhất. Chọn bưu kiện {target_parcel['label']}.")
    else:
        # Nếu có nhiều bưu kiện cao bằng nhau, chọn cái xa nhất (Y lớn nhất)
        print(f"\n⚠️ Có {len(top_group)} bưu kiện cao bằng nhau. Áp dụng luật 'xa robot hơn' (Y lớn nhất)...")
        # Sắp xếp nhóm này theo tọa độ Y giảm dần
        top_group.sort(key=lambda item: item['center_3d'][1], reverse=True)
        target_parcel = top_group[0]
        print(f"✨ Đã chọn bưu kiện {target_parcel['label']} vì có Y lớn nhất ({target_parcel['center_3d'][1]:.4f}).")

# --- KẾT QUẢ ---
if target_parcel:
    print("\n--- BƯU KIỆN MỤC TIÊU CUỐI CÙNG ---")
    print(f"  - Label: {target_parcel['label']}")
    print(f"  - Tọa độ tâm 3D: {np.round(target_parcel['center_3d'], 4)}")

📦 Danh sách bưu kiện ứng viên sau khi sắp xếp theo độ cao (Z tăng dần):
  - Bưu kiện 77: Z = 1.1698
  - Bưu kiện 54: Z = 1.1731
  - Bưu kiện 95: Z = 1.1744
  - Bưu kiện 62: Z = 1.1750
  - Bưu kiện 60: Z = 1.1930
  - Bưu kiện 58: Z = 1.1953
  - Bưu kiện 61: Z = 1.1961
  - Bưu kiện 50: Z = 1.1969
  - Bưu kiện 52: Z = 1.1970
  - Bưu kiện 86: Z = 1.1970
  - Bưu kiện 111: Z = 1.1970
  - Bưu kiện 101: Z = 1.1973
  - Bưu kiện 81: Z = 1.1976
  - Bưu kiện 45: Z = 1.1977
  - Bưu kiện 65: Z = 1.1977
  - Bưu kiện 85: Z = 1.1978
  - Bưu kiện 84: Z = 1.1980
  - Bưu kiện 53: Z = 1.1980
  - Bưu kiện 102: Z = 1.1982
  - Bưu kiện 79: Z = 1.1983
  - Bưu kiện 112: Z = 1.1984
  - Bưu kiện 96: Z = 1.1985
  - Bưu kiện 114: Z = 1.1986
  - Bưu kiện 67: Z = 1.1986
  - Bưu kiện 119: Z = 1.1990
  - Bưu kiện 92: Z = 1.1993
  - Bưu kiện 73: Z = 1.1995
  - Bưu kiện 76: Z = 1.1997
  - Bưu kiện 120: Z = 1.2002
  - Bưu kiện 117: Z = 1.2007
  - Bưu kiện 118: Z = 1.2049

⚠️ Có 3 bưu kiện cao bằng nhau. Áp dụng luật 'xa r